# 문제1.
*출처 : 롯데멤버스, L.pay|L.POINT, 제6회 L.POINT Big Data Competition*

### 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

In [2]:
cs = pd.read_csv('L사_고객정보.csv')
gd = pd.read_csv('L사_상품정보.csv')
tr = pd.read_csv('L사_거래정보.csv')

gd.pd_c = gd.pd_c.astype(str).str.zfill(4) # 상품과 거래 데이터의 pd_c 컬럼의 데이터 유형이 서로 달라 맞춰줌. 자리수까지도 맞춰줘야 함. 
df = pd.merge(tr, cs).merge(gd, on='pd_c')
df.de_dt = df.de_dt.astype(str).astype('datetime64') # 날짜정보를 담고 있는 de_dt 컬럼의 데이터 유형이 정수이기 때문

df.head()

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct,clnt_gender,clnt_age,clac_nm1,clac_nm2,clac_nm3
0,21922,104999,3,A03,0182,2019-09-20,12:41,10900,1,F,50,Chilled Foods,Chilled Instant Foods,Chilled Soups
1,39423,105124,10,A03,0182,2019-09-20,17:26,21800,2,F,50,Chilled Foods,Chilled Instant Foods,Chilled Soups
2,39423,89469,2,A03,0182,2019-09-01,03:32,25800,2,F,50,Chilled Foods,Chilled Instant Foods,Chilled Soups
3,39423,88436,1,A03,0182,2019-08-30,17:15,25800,2,F,50,Chilled Foods,Chilled Instant Foods,Chilled Soups
4,18362,50872,3,A03,0182,2019-07-15,09:46,31600,4,F,40,Chilled Foods,Chilled Instant Foods,Chilled Soups


In [3]:
#gd['clac_nm3'].value_counts()
df['pd_c']=df['pd_c'].astype(int)

### 1) ratings_matrx 생성하기
- 행은 clnt_id(고객ID), 열은 clac_nm3(상품 소분류명)로, vales='pd_c'(상품 소분류코드)로 pivoting 수행. 구매가 없는 값(NaN)은 모두 0으로 변환

In [4]:
rating_matrx=df.pivot_table('pd_c',index='clnt_id',columns='clac_nm3')
rating_matrx=rating_matrx.fillna(0)
rating_matrx.head()

clac_nm3,,Accessory Bags,Accident Prevention Equipment,Adhesive Tapes,Adhesives,Adult's Bed Covers and Skirts,Adult's Diapers,Adult's Disposable Briefes,Adults' Bed Fillings,Adults' Bedding Sets,...,Yoga / Pilates Clothing,Yoga Mats / Exercise Mats,Yoghurts / Chunggukjang Makers,Yogurt Drinks,Young Pumpkins,Young Radishes,Yuzus / Quinces,Zipper Poly Bags / Plastic Bags,kelp,life Vests / Safety Accessories
clnt_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1573.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1573.0,0.0,0.0,0.0,0.0,0.0


### 2) 코사인 유사도 구하기
- cosine_similarity를 사용하여 코사인 유사도를 구하고 데이터프레임 형태로 merchan_sim_df에 저장하세요

In [5]:
# ratings_matrix의 행과 열의 위치를 변경
rating_matrx=rating_matrx.transpose()
rating_matrx.head()

clnt_id,2,9,12,20,23,24,29,38,40,41,...,72333,72340,72342,72356,72361,72373,72400,72410,72423,72424
clac_nm3,,,,,,,,,,,,,,,,,,,,,
,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Accessory Bags,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Accident Prevention Equipment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Adhesive Tapes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Adhesives,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
temp=cosine_similarity(rating_matrx,rating_matrx)
print(temp.shape)
merchan_sim_df=pd.DataFrame(temp,index=rating_matrx.index,columns=rating_matrx.index)
merchan_sim_df

(1662, 1662)


clac_nm3,,Accessory Bags,Accident Prevention Equipment,Adhesive Tapes,Adhesives,Adult's Bed Covers and Skirts,Adult's Diapers,Adult's Disposable Briefes,Adults' Bed Fillings,Adults' Bedding Sets,...,Yoga / Pilates Clothing,Yoga Mats / Exercise Mats,Yoghurts / Chunggukjang Makers,Yogurt Drinks,Young Pumpkins,Young Radishes,Yuzus / Quinces,Zipper Poly Bags / Plastic Bags,kelp,life Vests / Safety Accessories
clac_nm3,,,,,,,,,,,,,,,,,,,,,
,1.000000,0.060193,0.000000,0.000000,0.000000,0.0,0.000000,0.077152,0.0,0.000000,...,0.000000,0.000000,0.0,0.037389,0.039820,0.000000,0.038236,0.040161,0.097590,0.000000
Accessory Bags,0.060193,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.017025,...,0.000000,0.000000,0.0,0.045011,0.019175,0.021979,0.000000,0.038678,0.000000,0.000000
Accident Prevention Equipment,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.015264,0.016256,0.000000,0.000000,0.032791,0.000000,0.000000
Adhesive Tapes,0.000000,0.000000,0.000000,1.000000,0.018359,0.0,0.000000,0.000000,0.0,0.000000,...,0.036298,0.000000,0.0,0.054573,0.060889,0.025380,0.031891,0.083741,0.000000,0.000000
Adhesives,0.000000,0.000000,0.000000,0.018359,1.000000,0.0,0.000000,0.000000,0.0,0.012451,...,0.045980,0.000000,0.0,0.049379,0.059601,0.000000,0.040398,0.063648,0.025777,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Young Radishes,0.000000,0.021979,0.000000,0.025380,0.000000,0.0,0.000000,0.000000,0.0,0.034427,...,0.000000,0.000000,0.0,0.077365,0.106628,1.000000,0.013962,0.053771,0.035635,0.000000
Yuzus / Quinces,0.038236,0.000000,0.000000,0.031891,0.040398,0.0,0.000000,0.000000,0.0,0.032444,...,0.000000,0.000000,0.0,0.071481,0.054812,0.013962,1.000000,0.036854,0.000000,0.041885
Zipper Poly Bags / Plastic Bags,0.040161,0.038678,0.032791,0.083741,0.063648,0.0,0.026774,0.024788,0.0,0.030291,...,0.027965,0.011593,0.0,0.176186,0.194037,0.053771,0.036854,1.000000,0.031354,0.000000


### 3) 특정 상품 유사도 추출
- 'Yogurt Drinks' 상품과 유사도가 가장 높은 상위 5개 상품 추출

In [7]:
merchan_sim_df['Yogurt Drinks'].sort_values(ascending=False)[1:6]

clac_nm3
General Snacks        0.482155
Fresh Milk            0.482012
Chicken Eggs          0.479976
Spoon Type Yogurts    0.477880
Tofu                  0.470498
Name: Yogurt Drinks, dtype: float64

### 4) 전체 상품에 대한 상품 코드 예측
- predic_trating 함수를 정의하고, ratings_pred_matrix에 데이터 프레임 형태로 저장하여 출력하세요.

In [8]:
def predic_trating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [9]:
ratings_pred = predic_trating(rating_matrx.T.values, merchan_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=rating_matrx.columns,
                                   columns=rating_matrx.index)
ratings_pred_matrix.head()

clac_nm3,,Accessory Bags,Accident Prevention Equipment,Adhesive Tapes,Adhesives,Adult's Bed Covers and Skirts,Adult's Diapers,Adult's Disposable Briefes,Adults' Bed Fillings,Adults' Bedding Sets,...,Yoga / Pilates Clothing,Yoga Mats / Exercise Mats,Yoghurts / Chunggukjang Makers,Yogurt Drinks,Young Pumpkins,Young Radishes,Yuzus / Quinces,Zipper Poly Bags / Plastic Bags,kelp,life Vests / Safety Accessories
clnt_id,,,,,,,,,,,,,,,,,,,,,
2,4.831164,4.560647,6.416234,4.577704,4.898771,8.610434,4.387551,4.465160,6.507797,6.352981,...,6.998340,2.678255,11.569932,8.382092,7.913181,6.038875,5.038979,6.492904,5.110838,7.538719
9,47.448659,45.106788,33.036319,44.406971,45.973919,30.871531,37.607266,58.273037,29.945666,46.096093,...,31.749175,49.568261,65.198679,73.319585,81.799391,63.548312,55.866569,57.236991,66.293925,54.770927
12,1.093866,0.905391,0.779759,1.076255,1.056596,0.334556,0.690932,1.320975,1.025912,1.193666,...,1.132619,1.098245,0.000000,1.709218,1.708081,1.553965,1.463108,1.384404,1.389942,1.451028
20,43.265502,34.284748,21.667229,34.938816,36.065744,11.677502,31.046046,52.417320,58.539267,43.630520,...,28.640368,30.130583,0.000000,61.787265,77.395871,73.290302,46.545577,51.806727,76.191556,48.228501
23,55.390552,42.988858,52.840703,46.506719,48.634422,25.538480,41.383886,55.211242,49.015010,53.804619,...,43.284281,48.482369,5.816373,75.618219,93.300395,79.665458,56.261992,60.975928,90.198992,59.044773


### 5) 전체 상품에 대한 상품 코드 예측 평가
- get_mse 함수를 생성하여, 전체 상품에 대한 mse를 구하세요.

In [10]:
from sklearn.metrics import mean_squared_error
def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)
print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, rating_matrx.T.values))

아이템 기반 모든 인접 이웃 MSE:  795517.5385679784


## 문제 2. 특정 상품과 가장 비슷한 유사도를 가지는 영화에 대해서만 유사도 벡터 적용 (1번과 연결)

### 1) 특정 상품과 가장 비슷한 유사도를 가지는 상품을 10개 나열하는 함수 만들기.

In [11]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=10):
    pred = np.zeros(ratings_arr.shape)

    for col in range(ratings_arr.shape[1]):
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n - 1:-1]]
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))
    return pred

### 2) 위 문제에서 만든 함수를 이용해 예측 구매율을 계산하고, 실제 구매율과의 MSE 구하기.

In [12]:
ratings_pred = predict_rating_topsim(rating_matrx.T.values, merchan_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, rating_matrx.T.values))

<ipython-input-11-27564a515fc0>:7: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
<ipython-input-11-27564a515fc0>:8: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))


아이템 기반 인접 TOP-20 이웃 MSE:  311203.28417414066


### 3) 고객번호 9번에 대해 상품 추천하기 (clnt_id=9)
- clnt_id =9 가 구매한 상품 상품 나열
- 고객이 구매하지 않은 상품을 리스트 객체로 반환하는 함수 생성 
- 위에서 만든 두 함수를 이용해 최종적으로 고객에게 상품을 추천하는 함수 만들기.

In [14]:
def get_unbought_items(ratings_matrix, userId):
    user_score = ratings_matrix.loc[:,userId]
    already_bought = user_score[user_score > 0].index.tolist()
    movies_list = ratings_matrix.index.tolist()
    unbought_list = [movie for movie in movies_list if movie not in already_bought]
    return unbought_list

In [15]:
def recomm_items_by_userid(pred_df, userId, unbought_list, top_n=10):
    recomm_movies = pred_df.loc[userId,unbought_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [16]:
unbought_list = get_unbought_items(rating_matrx, 9)
recomm_items = recomm_items_by_userid(ratings_pred_matrix, 9, unbought_list, top_n=10)

In [17]:
recomm_items

clac_nm3
Kids' Beds                               116.492004
Sun Shelters                             112.429100
Window Air Conditioners                  108.773597
Green Lavers                             102.892176
Vacuum-insulated Lunch Boxes              96.148753
Australian Imported Beefs - Chunks        86.384953
Chilled Chicken, Jokbal and Pork Feet     85.427872
Canned Sauries / Mackerels                83.928437
Frozen Tteokbokkis                        83.275434
Processed Quail Eggs                      82.951626
Name: 9, dtype: float64

### 4) 행렬분해를 이용해 잠재 요인 협업 필터링 직접 구현하기.
- SGD기법을 사용하는 행렬 분해 함수 만들기
- 고객-상품 구매 행렬 다시 만들기
- 만들어진 행렬을 행렬분해하기 (steps=200, 잠재 요인 차원 = 30, 학습률과 L2 Regularization 계수 = 0.01)
- 9번 고객에 대해 

In [18]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape
    np.random.seed(1)
    P = np.random.normal(scale=1. / K, size=(num_users, K))
    Q = np.random.normal(scale=1. / K, size=(num_items, K))

    break_count = 0

    non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

    for step in range(steps):
        for i, j, r in non_zeros:
            eij = r - np.dot(P[i, :], Q[j, :].T)
            P[i, :] = P[i, :] + learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])
            Q[j, :] = Q[j, :] + learning_rate * (eij * P[i, :] - r_lambda * Q[j, :])

        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0:
            print("### iteration step : ", step, " rmse : ", rmse)

    return P, Q

In [19]:
def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P, Q.T)

    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]

    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [20]:
P, Q = matrix_factorization(rating_matrx.T.values, K=30, steps=200, learning_rate=0.01, r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=rating_matrx.columns,
                                   columns=rating_matrx.index)

<ipython-input-18-cc551322dbde>:15: RuntimeWarning: overflow encountered in multiply
  Q[j, :] = Q[j, :] + learning_rate * (eij * P[i, :] - r_lambda * Q[j, :])
<ipython-input-18-cc551322dbde>:14: RuntimeWarning: overflow encountered in multiply
  P[i, :] = P[i, :] + learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])
<ipython-input-18-cc551322dbde>:14: RuntimeWarning: invalid value encountered in add
  P[i, :] = P[i, :] + learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
unbought_list = get_unbought_items(rating_matrx, 9)
recomm_items = recomm_items_by_userid(ratings_pred_matrix, 9, unbought_list, top_n=10)

## 문제3) Surprise를 활용한 책 추천 시스템 구축

In [21]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
import pandas as pd
from surprise import Reader, Dataset

### 3-1) book_ratings 데이터를 불러온 후, 아래의 형식과 같이 출력해주세요.
- Total number of ratings : xxx
- Number of unique users : xxx
- Number of unique movies : xxx

In [22]:
ratings=pd.read_csv('book_ratings.csv')
ratings.head()

,user_id,book_id,rating
0,1,1180,4
1,1,4893,3
2,1,6285,4
3,2,8034,4
4,2,8855,5


In [23]:
print("Total number of ratings : ",len(ratings))
print("Number of unique users : ",len(ratings['user_id'].unique()))
print("Number of unique books : ",len(ratings['book_id'].unique()))

Total number of ratings :  981756
Number of unique users :  53424
Number of unique books :  10000


### books 데이터를 불러온 후, id, authors, title에 해당하는 열만 추출하여 books에 저장해주세요

In [24]:
books=pd.read_csv("books.csv",encoding='latin1')
books=books[["id","authors","original_title"]]
books

,id,authors,original_title
0,1,Suzanne Collins,The Hunger Games
1,2,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Philosopher's Stone
2,3,Stephenie Meyer,Twilight
3,4,Harper Lee,To Kill a Mockingbird
4,5,F. Scott Fitzgerald,The Great Gatsby
...,...,...,...
9990,9996,Ilona Andrews,Bayou Moon
9991,9997,Robert A. Caro,Means of Ascent
9992,9998,Patrick O'Brian,The Mauritius Command
9993,9999,Peggy Orenstein,Cinderella Ate My Daughter: Dispatches from th...


### 3-2) Ratings 데이터프레임에서 surprise 데이터 세트로 로딩하여 RMSE를 출력해주세요
- 알고리즘은 SVD, n_factors=30
- test_size=0.25, random_state=0
- movies_ratings_small에서 rating의 범위를 기반으로 rating_scale 설정

In [25]:
import pandas as pd
from surprise import Reader, Dataset
reader=Reader(rating_scale=(1,5))
data = Dataset.load_from_df(ratings, reader)

In [26]:
trainset, testset=train_test_split(data,test_size=0.25,random_state=0)
algo = SVD(random_state=0,n_factors=30)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8405


0.8405238041196965

### 3-3) GridSearchCV를 이용하여 최적 파라미터를 산출하세요
- param_grid={'n_epochs':[15,20,25],'n_factors':[50,80,100,120]}

In [27]:
from surprise.model_selection import GridSearchCV
param_grid = {'n_epochs': [15,20,25], 'n_factors': [50, 80, 100, 120]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.844672723707491
{'n_epochs': 25, 'n_factors': 50}


In [31]:
data = Dataset.load_from_df(ratings, reader)
algo = SVD(n_factors=50, random_state=0, n_epochs=25)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions, verbose=True)

RMSE: 0.8408


0.8408161047920769

### 3-4) DatasetAutoFolds를 활용하여 개인 책 추천 시스템을 구축하세요

- 1)book_ratings에서 헤드를 제거하여 새로운 파일을 생성해주세요. (book_ratings_noh.csv)

In [32]:
ratings.to_csv('book_ratings_noh.csv',index=False,header=False)

- 2) book_ratings_noh 파일 기반으로 전체 데이터를 SVD로 학습 시켜주세요. (n_epochs와 n_factors는 GridSearchCV를 기반으로 나온 최적 파라미터, random_state=0)

In [35]:
from surprise.dataset import DatasetAutoFolds
reader = Reader(line_format='user item rating', sep=',', rating_scale=(1, 5))
data_folds = DatasetAutoFolds(ratings_file='book_ratings_noh.csv', reader=reader)

In [36]:
trainset = data_folds.build_full_trainset()

algo = SVD(n_epochs=25, n_factors=50, random_state=0)
algo.fit(trainset)

- 교재에서 사용한 get_unseen_surprise와 recomm_movie_by_surprise 함수를 기반으로 userID=10인 사용자에게 top10 개의 책 리스트와 예측 평점을 산출하세요.

In [98]:
def get_unseen_surprise(ratings, books, userId):
    seen_books = ratings[ratings['user_id'] == userId]['book_id'].tolist()

    total_books = books['id'].tolist()

    unseen_books = [book for book in total_books if book not in seen_books]
    print('평점 매긴 책 수:', len(seen_books), '추천대상 책 수:', len(unseen_books), \
          '전체 책 수:', len(total_books))

    return unseen_books
unseen_books = get_unseen_surprise(ratings, books, 10)

평점 매긴 책 수: 16 추천대상 책 수: 9979 전체 책 수: 9995


In [99]:
def recomm_book_by_surprise(algo, userId, unread_books, top_n=10):
    predictions = [algo.predict(str(userId), str(item)) for item in unread_books]
    
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    
    top_book_ids = [ int(pred.iid) for pred in top_predictions]
    top_book_rating = [ pred.est for pred in top_predictions]
    top_book_titles = books[books.id.isin(top_book_ids)]['original_title']
    
    top_book_preds = [ (id, rating) for id, rating in 
                      zip(top_book_ids, top_book_rating)]
    return top_book_preds

top_book_preds = recomm_book_by_surprise(algo, 10, unseen_books, top_n=10)

top_list=books[books["id"].isin([i[0] for i in top_book_preds])]['original_title'].values
for top_list_name, top_book in zip(top_list,top_book_preds) :
    print(top_list_name,":", top_book[1])

A Game of Thrones : 4.891455596919947
Words of Radiance : 4.877056710164582
Jesus the Christ: A Study of the Messiah and His Mission according to Holy Scriptures both Ancient and Modern : 4.854118497561855
The Days Are Just Packed: A Calvin and Hobbes Collection : 4.849581190126291
The Authoritative Calvin and Hobbes : 4.843138740752508
The Indispensable Calvin and Hobbes: A Calvin and Hobbes Treasury : 4.8345439051958605
Homicidal Psycho Jungle Cat: A Calvin and Hobbes Collection : 4.831507374433395
Livro do Desassossego : 4.8278391038455615
Crush : 4.803944680284454
Humans of New York: Stories : 4.762878963076419
